In [1]:
import pandas as pd
import src.wrangle as wrangle
import src.features as features
import src.preprocessing as preprocessing

In [2]:
df = wrangle.get_space_data()

In [3]:
df.head()

,company_name,location,rocket_type,rocket_status,mission_cost,mission_status
date_time,,,,,,
2020-08-07 05:12:00+00:00,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,active,50.0,Success
2020-08-06 04:01:00+00:00,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...",Long March 2D | Gaofen-9 04 & Q-SAT,active,29.75,Success
2020-08-04 23:57:00+00:00,SpaceX,"Pad A, Boca Chica, Texas, USA",Starship Prototype | 150 Meter Hop,active,0,Success
2020-07-30 21:25:00+00:00,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,active,65.0,Success
2020-07-30 11:50:00+00:00,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA",Atlas V 541 | Perseverance,active,145.0,Success


In [4]:
df.to_csv('../data/time_series_space_data')

In [5]:
df.mission_status.value_counts()

Success              3879
Failure               339
Partial Failure       102
Prelaunch Failure       4
Name: mission_status, dtype: int64

> Let's make a new column that encodes the values from `mission_status` into a 1 or a 0 for success or failure

In [6]:
df = df.pipe(features.mission_result)

In [7]:
df.head()

,company_name,location,rocket_type,rocket_status,mission_cost,mission_status,mission_result
date_time,,,,,,,
2020-08-07 05:12:00+00:00,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,active,50.0,Success,1
2020-08-06 04:01:00+00:00,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...",Long March 2D | Gaofen-9 04 & Q-SAT,active,29.75,Success,1
2020-08-04 23:57:00+00:00,SpaceX,"Pad A, Boca Chica, Texas, USA",Starship Prototype | 150 Meter Hop,active,0,Success,1
2020-07-30 21:25:00+00:00,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,active,65.0,Success,1
2020-07-30 11:50:00+00:00,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA",Atlas V 541 | Perseverance,active,145.0,Success,1


# Preprocessing

## Splitting the data

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
train, test = train_test_split(df, test_size=.2, random_state=123)

In [10]:
train.shape

(3459, 7)

In [11]:
test.shape

(865, 7)

In [12]:
train.reset_index(inplace=True)

## Encoding

In [13]:
from sklearn.preprocessing import LabelEncoder

In [14]:
cols_to_encode = ['company_name', 'location', 'rocket_type']

In [15]:
for col in cols_to_encode:
    encoder = LabelEncoder()
    df[col] = encoder.fit_transform(df[col])

In [16]:
train, split = preprocessing.split_data(df)

In [17]:
train = preprocessing.encode_data(train)
test = preprocessing.encode_data(test)

/Users/shayaltshue/projects/Final-Frontier/shays-stuff/src/preprocessing.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = encoder.fit_transform(df[col])


# Modeling

In [18]:
X = train[['company_name', 'location', 'rocket_type', 'mission_cost']]
y = train.mission_result

## Tree Model

In [19]:
from sklearn.tree import DecisionTreeClassifier

In [20]:
tree = DecisionTreeClassifier(max_depth=3, random_state=123)

In [21]:
tree.fit(X, y)

ValueError: could not convert string to float: '1,160.0 '